In [2]:
import pandas as pd

In [3]:
# 读取csv
cards = pd.read_csv("card_list.csv")
cards.shape

(160, 11)

In [ ]:
# 读取前五行
cards.head()

In [ ]:
# 保存csv
cards.to_csv("cardscopy.csv")

In [ ]:
# 访问DataFrame中的数据--列
# 属性形式
cards.Name

In [ ]:
# 字典形式
cards["Name"]

In [ ]:
# 访问列中的某个值
cards.Name[0]

In [ ]:
# 数字索引
# 行索引 iloc 先行后列，和普通python相反
cards.iloc[0]
# 单独的 : 意味着所有，[1:3]左闭右开区间
cards.iloc[:3] # [0, 3)
cards.iloc[1:3] # [1, 3)
cards.iloc[-5:] # 倒数第5到最后
cards.iloc[[1, 5, 15, 159], [0, 1]] # 用列表指定行，必须指定列，列表套列表第一个指定行，第二个不写默认全选

In [ ]:
# 标签索引 loc
cards.loc[0, 'Name']
cards.loc[[1,2,3]]

iloc和loc的区别
* iloc使用的是python标准库中的索引机制，即左闭右开
* loc使用的是pandas的索引，使用左闭右闭区间
* loc可以使用字符串等其他类型\["Apple":"Potato"\]，选择从A到P之间所有的水果

In [ ]:
# 修改索引列，选择其他更好的列作为索引
cards.set_index("Name")
cards.head()

In [ ]:
# 条件索引 用于在DataFrame中找符合条件的数据
# 返回一个bool列表，可用于索引，True对应行的数据显示，False行不显示
blist = cards.Type == "creature"
cards[blist]
# 用 & 或 | 同时使用多个条件
cards[blist & (cards.Attack >= 7)]

In [ ]:
# 所有isin来选择符合列表中值的行
cards[cards.Cost.isin([8,9,12])]

In [ ]:
# 判断有没有空值
cards.loc[cards.Descriptions.isnull()]
cards.loc[cards.Descriptions.notnull()]

In [ ]:
# 给DataFrame赋值
# 给定常量，则一列都被赋予该值
cards.Abilities = "-0--0-"
# 给定迭代器
cards.Abilities = range(len(cards), 0, -1)
cards.head()

In [ ]:
# 统计函数
# 数值型
cards.Attack.describe()
cards.Attack.mean()

In [ ]:
# 字符型总统计数据
cards.Type.describe()
# 不重复的值列表
cards.Type.unique()
# 不重复的值的频率列表
cards.Type.value_counts()

Map  
* 映射，将一个Series中的数x修改为f(x)
* lambda p : f(p)  其中p是Series中单个的输入，f(p)是你修改后的输出
* map() 用于Series对象
* apply() 用于DataFrame对象，对每一行应用
* map和apply都没有修改原来的对象，而是返回了新的修改过的对象
* pandas有内建的map,apply方法，如cards.Attack - cards.Attack.mean,cards.Number + "-" + cards.Name

In [ ]:
cmean = cards.Attack.mean()
cards.Attack.map(lambda p : p-cmean)

In [10]:
def remean(row):
    row.Attack = row.Attack - cards.Attack.mean()
    return row
# axis="columns" 输入是行，remean修改行的输入
cards.apply(remean,axis="columns")

,Number,Name,Type,Cost,Attack,Defense,Abilities,PlayerHP,EnemyHP,CardDraw,Descriptions
0,1,Slimer,creature,1,-0.875,1,------,1,0,0,Summon: You gain 1 health.
1,2,Scuttler,creature,1,-1.875,2,------,0,-1,0,Summon: Deal 1 damage to your opponent.
2,3,Beavrat,creature,1,-0.875,2,------,0,0,0,NaN
3,4,Plated Toad,creature,2,-1.875,5,------,0,0,0,NaN
4,5,Grime Gnasher,creature,2,1.125,1,------,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
155,156,Major Life Steal Potion,itemBlue,3,-2.875,0,------,3,-3,0,Deal 3 damage to your opponent and gain 3 health.
156,157,Life Sap Drop,itemBlue,3,-2.875,-1,------,1,0,1,Deal 1 damage. gain 1 health and draw a card.
157,158,Tome of Thunder,itemBlue,3,-2.875,-4,------,0,0,0,Deal 4 damage.
158,159,Vial of Soul Drain,itemBlue,4,-2.875,-3,------,3,0,0,Deal 3 damage and gain 3 health.


In [9]:
# 注意返回值不同，返回的对象就不同，返回值是row，则返回DataFrame，返回具体值，则返回Series
def star(row):
    if (row.Cost > 7):
        return 3
    elif (row.Cost > 4):
        return 2
    else:
        return 1
cards.apply(star,axis="columns")   
# axis="index" 则输入是列，需要写修改列输入的方法

0      1
1      1
2      1
3      1
4      1
      ..
155    1
156    1
157    1
158    1
159    1
Length: 160, dtype: int64

In [ ]:
# 注意类型
cards.Name + "-" + cards.Descriptions

In [ ]:
# sum()统计总数
# in 用于判断序列中包不包含指定值，返回bool 
n_me = cards.Name.map(lambda desc:"me" in desc).sum() 
n_s = cards.Name.map(lambda desc:"s" in desc).sum() 
pd.Series([n_me, n_s], index=["meeee", "ssss"])

groupby()
* 按参数分组，所有值相同的分为一个小组，每个小组可看作DataFrame的一个切片(每个小组都是一个小的DataFrame)
* 对groupby()返回的小组(DataFrameGroupBy对象)可应用apply方法
* 可以使用多个参数分组，使用列表指定多个参数，和mysql里面的复合key类似
* agg()函数用于分组后同时对小组进行多种统计，使用列表作为参数

In [ ]:
# 实现value_count()
cards.groupby("Type").Type.count()
# 使用apply()选择每个Type的第一次出现的数据
cards.groupby("Type").apply(lambda df : df.iloc[0])

In [ ]:
# 使用多个列分组
cards.groupby(["Type", "Cost"]).apply(lambda df : df.loc[df.Cost.idxmax()])

In [ ]:
# agg使用示例 同时计算长度，最小值，最大值
atk_statistics = cards.groupby(["Type", "Cost"]).Attack.agg([len, max, min])
atk_statistics

多重索引
* type(atk.index) #pandas.core.indexes.multi.MultiIndex
* 和常规索引不同，有分层结构，有相应的处理方法
* 多重索引需要多个键值才能索引到数据
* 具体用法比较复杂

In [ ]:
atk = cards.groupby(["Type", "Cost"]).Attack.agg([len])
# 将多重索引重设为普通索引
atk.reset_index()

排序
* 通常数据都是按索引排序的
* 使用sort_values()方法来按数据排序
    * by="col" 指定按那一列排序
    * ascending=False 使用降序(默认是升序)
* sort_index()按索引排序
* 和groupby()一样，可以使用多个键值来索引

In [ ]:
atk.sort_values(by="len", ascending=False)
atk_statistics.sort_values(by=["len", "max"])

In [ ]:
cards.groupby("Cost").agg([len])

查看类型
* Series 用dtype属性
* DataFrame 用dtypes属性查看所有列的属性
* 只用字符串的是object属性
* 使用astype()进行类型转换
* Series和DataFrame的索引有自己的属性

In [ ]:
cards.dtype

缺失值处理
* 缺失值用NaN(not a number)表示。由于技术原因，所有的NaN都是float64类型
* 判断有没有缺失值用pd.isnull()或相对的pd.notnull()
* 替换缺失值用fillna()
* 修改值用replace()

In [ ]:
cards[pd.isnull(cards.Descriptions)]
cards[pd.notnull(cards.Descriptions)]

In [ ]:
# fillna()
cards.Descriptions.fillna("No Descriptions")

In [ ]:
# replace
cards.Name.replace("Slimer", "史莱姆")

In [ ]:
# 统计缺失值个数
# 法一：bool索引找到有缺失数据的行
missing = cards[cards.Descriptions.isnull()]
len(missing)
# 法二：用isnull().sum() True被视为1，False被视为0，故sum()可用
pd.isnull(cards.Descriptions).sum()

修改列名，行索引名
* rename(columns={"OldName":"NewName", ...}, index={"old":"new"})
* 列名和行索引也可以有名字，用rename_axis("Attr", axis="columns")

In [20]:
cards.rename_axis("attr", axis="columns").rename_axis("#N", axis="rows")

attr,Number,Name,Type,Cost,Attack,Defense,Abilities,PlayerHP,EnemyHP,CardDraw,Descriptions
#N,,,,,,,,,,,
0,1,Slimer,creature,1,2,1,------,1,0,0,Summon: You gain 1 health.
1,2,Scuttler,creature,1,1,2,------,0,-1,0,Summon: Deal 1 damage to your opponent.
2,3,Beavrat,creature,1,2,2,------,0,0,0,NaN
3,4,Plated Toad,creature,2,1,5,------,0,0,0,NaN
4,5,Grime Gnasher,creature,2,4,1,------,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
155,156,Major Life Steal Potion,itemBlue,3,0,0,------,3,-3,0,Deal 3 damage to your opponent and gain 3 health.
156,157,Life Sap Drop,itemBlue,3,0,-1,------,1,0,1,Deal 1 damage. gain 1 health and draw a card.
157,158,Tome of Thunder,itemBlue,3,0,-4,------,0,0,0,Deal 4 damage.


合并
* concat(\["dfA", "dfB"\]) 两个相同的列表，连在一起，作为行增加
* left.join(right, lsuffix="_L", rsuffix="_R") 作为列增加，right加入left，left中的列加_L后缀，right中的列加_R后缀

In [32]:
news = pd.read_csv("new.csv")
pd.concat([cards, news])

,Number,Name,Type,Cost,Attack,Defense,Abilities,PlayerHP,EnemyHP,CardDraw,...,name,type,cost,attack,defense,abilities,playerHP,enemyHP,cardDraw,descriptions
0,1,Slimer,creature,1.0,2.0,1.0,------,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Scuttler,creature,1.0,1.0,2.0,------,0.0,-1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Beavrat,creature,1.0,2.0,2.0,------,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Plated Toad,creature,2.0,1.0,5.0,------,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Grime Gnasher,creature,2.0,4.0,1.0,------,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SB,creature,1.0,999.0,999.0,------,1.0,0.0,0.0,Summon: You gain 999 health.
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SB,creature,1.0,999.0,999.0,------,1.0,0.0,0.0,Summon: You gain 999 health.
2,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SB,creature,1.0,999.0,999.0,------,1.0,0.0,0.0,Summon: You gain 999 health.
3,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SB,creature,1.0,999.0,999.0,------,1.0,0.0,0.0,Summon: You gain 999 health.


In [33]:
# 多的加少的会补缺失值，少的加多的，多的被截断。有列名相同时，加后缀以区分
cards.join(news, lsuffix="_L", rsuffix="_R")
news.join(cards, lsuffix="_L", rsuffix="_R")
# 设置以那一列为索引合并，除索引外，没有不同的列名时，可以不加后缀
news.set_index("Number").join(cards.set_index("Number"))

,name,type,cost,attack,defense,abilities,playerHP,enemyHP,cardDraw,descriptions,Name,Type,Cost,Attack,Defense,Abilities,PlayerHP,EnemyHP,CardDraw,Descriptions
Number,,,,,,,,,,,,,,,,,,,,
1,SB,creature,1,999,999,------,1,0,0,Summon: You gain 999 health.,Slimer,creature,1,2,1,------,1,0,0,Summon: You gain 1 health.
2,SB,creature,1,999,999,------,1,0,0,Summon: You gain 999 health.,Scuttler,creature,1,1,2,------,0,-1,0,Summon: Deal 1 damage to your opponent.
24,SB,creature,1,999,999,------,1,0,0,Summon: You gain 999 health.,Exploding Skitterbug,creature,1,1,1,------,0,-1,0,Summon: Deal 1 damage to your opponent.
25,SB,creature,1,999,999,------,1,0,0,Summon: You gain 999 health.,Spiney Chompleaf,creature,2,3,1,------,-2,-2,0,Summon: Deal 2 damage to each player.
26,SB,creature,1,999,999,------,1,0,0,Summon: You gain 999 health.,Razor Crab,creature,2,3,2,------,0,-1,0,Summon: Deal 1 damage to your opponent.
